# Spoonacular API Calls

___

### Table Of Contents

1. 
2. 
3. 
4. 
5. 
6. 
7. 
8. 
9. 
10. 

### 1. Import Libraries

In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd
import requests
import json
import sklearn
from scipy import stats
from scipy.stats import norm
from sklearn.utils import resample
import pickle
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats
from wordcloud import WordCloud
import random
from collections import Counter

___

### 2. Expand maximum range of rows

Lots of incoming data from the Spoonacular API.<br>
It is probably a good idea to expand the max range of rows/columns/width to view in Pandas.<br>
Use <b>pd.set_option()</b>

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

___

### 3. Understanding the Spoonacular API documentation

Documentation for the Wordtracker API:<br> 
https://spoonacular.com/food-api/docs

#### Identify All GET Endpoints (No PATCH, POST, PUT, or DELETE)

- <b>RECIPES</b>
    - recipes/search
    - /recipes/findByNutrients
    - /recipes/findByIngredients
    - /recipes/complexSearch
    - /recipes/{id}/information
    - /recipes/informationBulk
    - /recipes/{id}/similar
    - /recipes/random
    - /recipes/autocomplete
    - /recipes/{id}/equipmentWidget.json
    - /recipes/{id}/priceBreakdownWidget.json
    - /recipes/{id}/ingredientWidget.json
    - /recipes/{id}/nutritionWidget.json
    - /recipes/{id}/analyzedInstructions
    - /recipes/extract
    - /recipes/{id}/summary
    - /recipes/queries/analyze
    - /recipes/guessNutrition
    
    
- <b>INGREDIENTS</b>
    - /food/ingredients/{id}/information
    - /recipes/convert
    - /food/ingredients/autocomplete
    - /food/ingredients/substitutes
    - /food/ingredients/{id}/substitutes
    
    
- <b>PRODUCTS</b>
    - /food/products/search
    - /food/products/upc/{upc}
    - /food/products/{id}
    - /food/products/upc/{upc}/comparable
    - /food/products/suggest
    
    
- <b>MENU ITEMS</b>
    - /food/menuItems/search
    - /food/menuItems/{id}
    - /food/menuItems/suggest
    
    
- <b>MEAL PLANS</b>
    - /recipes/mealplans/generate
    
    
- <b>WINE</b>
    - /food/wine/dishes
    - /food/wine/pairing
    - /food/wine/description
    - /food/wine/recommendation
    
    
- <b>MISC</b>
    - /recipes/quickAnswer
    - /food/detect
    - /food/site/search
    - /food/videos/search
    - /food/jokes/random
    - /food/trivia/random
    - /food/converse
    - /food/converse/suggest
    
    
- <b>WIDGETS</b>
    - /recipes/{id}/ingredientWidget
    - /recipes/{id}/equipmentWidget
    - /recipes/{id}/priceBreakdownWidget
    - /recipes/{id}/nutritionWidget
    - /recipes/visualizeNutrition
    - /recipes/visualizePriceEstimator
    - /recipes/visualizeEquipment
    - /recipes/visualizeIngredients
    - /food/products/{id}/nutritionWidget
    - /food/menuItems/{id}/nutritionWidget
    - /recipes/visualizeRecipe

#### Identify <u>critical</u> GET endpoints and describe them

- <b>RECIPES</b>
    - <b>recipes/search</b>
        - Spoonacular recipe API includes over 360,000 recipes as well as an open source recipe database. Consider using the "Search Recipes Complex" endpoint for much more flexibility.
    - <b>/recipes/findByNutrients</b>
        - Find a set of recipes that adhere to the given nutritional limits. You may set limits for macronutrients (calories, protein, fat, and carbohydrate) and/or many micronutrients.
    - <b>/recipes/findByIngredients</b>
        - Ever wondered what recipes you can cook with the ingredients you have in your fridge or pantry? This endpoint lets you find recipes that either maximize the usage of ingredients you have at hand (pre shopping) or minimize the ingredients that you don't currently have (post shopping). Find recipes that use as many of the given ingredients as possible and require as few additional ingredients as possible. This is a "what's in your fridge" API endpoint.
    - <b>/recipes/complexSearch</b>
        - Search through hundreds of thousands of recipes using advanced filtering and ranking. NOTE: This method combines searching by query, by ingredients, and by nutrients into one endpoint.
    - <b>/recipes/{id}/information</b>
        - Use a recipe id to get full information about a recipe, such as ingredients, nutrition, diet and allergen information, etc.
    - <b>/recipes/informationBulk</b>
        - Get information about multiple recipes at once. This is equivalent to calling the Get Recipe Information endpoint multiple times, but faster.
    - <b>/recipes/{id}/similar</b>
        - Find recipes which are similar to the given one.
    - <b>/recipes/{id}/summary</b>
        - Automatically generate a short description that summarizes key information about the recipe.
    
    
- <b>INGREDIENTS</b>
    - <b>/food/ingredients/{id}/information</b>
        - Use an ingredient id to get all available information about an ingredient, such as its image and supermarket aisle.
    - <b>/food/ingredients/substitutes</b>
        - Search for substitutes for a given ingredient.
    
    
- <b>PRODUCTS</b>
    - <b>/food/products/search</b>
        - Search packaged food products, such as frozen pizza or Greek yogurt.
    - <b>/food/products/{id}</b>
        - Use a product id to get full information about a product, such as ingredients, nutrition, etc.
    
    
- <b>MENU ITEMS</b>
    - <b>/food/menuItems/search</b>
        - Search over 115,000 menu items from over 800 fast food and chain restaurants. For example, McDonald's Big Mac or Starbucks Mocha.
            - <b>/food/menuItems/{id}</b>
        - Use a menu item id to get all available information about a menu item, such as nutrition.
            - <b>/food/menuItems/suggest</b>
        - Generate suggestions for menu items based on a (partial) query. The matches will be found by looking in the title only.

___

### 4. Test A Call To Spoonacular API

#### Get a recipe

Test a call to the <b>/recipes/search</b> endpoint to make sure we can return a recipe.

In [3]:
response_1_name = requests.get('https://api.spoonacular.com/recipes/search?query=Kale+and+Quinoa+Salad&number=1&apiKey=c6943e3e749f4d5c8627c071b32cd36a')
example_recipe_name = response_1_name.json()
example_recipe_name

{'results': [{'id': 617707,
   'title': 'Quinoa & Kale Salad',
   'readyInMinutes': 20,
   'servings': 4,
   'image': 'Quinoa---Kale-Salad-617707.jpg',
   'imageUrls': ['Quinoa---Kale-Salad-617707.jpg']}],
 'baseUri': 'https://spoonacular.com/recipeImages/',
 'offset': 0,
 'number': 1,
 'totalResults': 57,
 'processingTimeMs': 72,
 'expires': 1579484832484,
 'isStale': False}

#### Get more detailed information about this recipe.

Test a call to the <b>/recipes/{id}/information</b> endpoint to make sure we can use the "id" number provided to return more detailed information about this "Kale and Quinoa Salad with Black Beans" recipe that we got above.

In [4]:
response_1_info = requests.get('https://api.spoonacular.com/recipes/592479/information?includeNutrition=true&apiKey=c6943e3e749f4d5c8627c071b32cd36a')
example_recipe_info = response_1_info.json()
example_recipe_info

{'vegetarian': True,
 'vegan': True,
 'glutenFree': True,
 'dairyFree': True,
 'veryHealthy': True,
 'cheap': False,
 'veryPopular': True,
 'sustainable': False,
 'weightWatcherSmartPoints': 6,
 'gaps': 'no',
 'lowFodmap': False,
 'ketogenic': False,
 'whole30': False,
 'preparationMinutes': 25,
 'cookingMinutes': 25,
 'sourceUrl': 'http://blog.fatfreevegan.com/2013/06/kale-and-quinoa-salad-with-black-beans.html',
 'spoonacularSourceUrl': 'https://spoonacular.com/kale-and-quinoa-salad-with-black-beans-592479',
 'aggregateLikes': 50078,
 'spoonacularScore': 100.0,
 'healthScore': 100.0,
 'creditsText': 'Fat Free Vegan',
 'sourceName': 'Fat Free Vegan',
 'pricePerServing': 101.23,
 'extendedIngredients': [{'id': 1022009,
   'aisle': 'Ethnic Foods;Spices and Seasonings',
   'image': 'chili-powder.jpg',
   'consitency': 'solid',
   'name': 'ancho chile powder',
   'original': '1 teaspoon ancho chile powder (or other pure chile powder, not a blend)',
   'originalString': '1 teaspoon ancho c

#### Explore what kind of information that we can get out of here.

In [5]:
len(example_recipe_info)

38

In [6]:
response_2_info = requests.get('https://api.spoonacular.com/recipes/667917/information?includeNutrition=true&apiKey=c6943e3e749f4d5c8627c071b32cd36a')
example_recipe_info2 = response_2_info.json()

In [7]:
df = pd.DataFrame(data = [example_recipe_info, example_recipe_info2])
df.head()

,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,weightWatcherSmartPoints,gaps,lowFodmap,ketogenic,whole30,preparationMinutes,cookingMinutes,sourceUrl,spoonacularSourceUrl,aggregateLikes,spoonacularScore,healthScore,creditsText,sourceName,pricePerServing,extendedIngredients,id,title,readyInMinutes,servings,image,imageType,nutrition,cuisines,dishTypes,diets,occasions,winePairing,instructions,analyzedInstructions
0,True,True,True,True,True,False,True,False,6,no,False,False,False,25.0,25.0,http://blog.fatfreevegan.com/2013/06/kale-and-...,https://spoonacular.com/kale-and-quinoa-salad-...,50078,100.0,100.0,Fat Free Vegan,Fat Free Vegan,101.23,"[{'id': 1022009, 'aisle': 'Ethnic Foods;Spices...",592479,Kale and Quinoa Salad with Black Beans,50,6,https://spoonacular.com/recipeImages/592479-55...,jpg,"{'nutrients': [{'title': 'Calories', 'amount':...",[],[side dish],"[gluten free, dairy free, lacto ovo vegetarian...",[],{},Instructions Heat a saucepan. Add the rinsed ...,"[{'name': '', 'steps': [{'number': 1, 'step': ..."
1,True,True,True,True,True,False,True,False,1,no,False,False,True,NaN,NaN,http://www.thegraciouspantry.com/clean-eating-...,https://spoonacular.com/cilantro-salsa-667917,251583,100.0,58.0,The Gracious Pantry,The Gracious Pantry,112.71,"[{'id': 11165, 'aisle': 'Produce;Spices and Se...",667917,Cilantro Salsa,20,2,https://spoonacular.com/recipeImages/667917-55...,jpg,"{'nutrients': [{'title': 'Calories', 'amount':...",[Mexican],[side dish],"[gluten free, dairy free, paleolithic, lacto o...",[],"{'pairedWines': ['pinot noir', 'riesling', 'sp...",Chop all your ingredients. Be sure you chop th...,"[{'name': '', 'steps': [{'number': 1, 'step': ..."


In [8]:
#this should be a dict since we used .json()
type(example_recipe_info)

dict

In [9]:
#what are the keys of our dictionary?
example_recipe_info_keys = list(example_recipe_info.keys())
example_recipe_info_keys

['vegetarian',
 'vegan',
 'glutenFree',
 'dairyFree',
 'veryHealthy',
 'cheap',
 'veryPopular',
 'sustainable',
 'weightWatcherSmartPoints',
 'gaps',
 'lowFodmap',
 'ketogenic',
 'whole30',
 'preparationMinutes',
 'cookingMinutes',
 'sourceUrl',
 'spoonacularSourceUrl',
 'aggregateLikes',
 'spoonacularScore',
 'healthScore',
 'creditsText',
 'sourceName',
 'pricePerServing',
 'extendedIngredients',
 'id',
 'title',
 'readyInMinutes',
 'servings',
 'image',
 'imageType',
 'nutrition',
 'cuisines',
 'dishTypes',
 'diets',
 'occasions',
 'winePairing',
 'instructions',
 'analyzedInstructions']

In [10]:
#The unique id for the recipe.
example_recipe_info['id']

592479

In [11]:
#Nutrition data for the recipe is pretty long, let's just get calories. 
print(example_recipe_info['nutrition']['nutrients'][0]['title'])

#Amount of calories that are in this recipe.
example_recipe_info['nutrition']['nutrients'][0]['amount']

Calories


242.6

In [12]:
len(example_recipe_info['instructions'].split())

186

In [13]:
# Num ingredients
print("Number of ingredients:", len(example_recipe_info['extendedIngredients']), '\n')

# List ingredients
ing_list = []
for index, val in enumerate(example_recipe_info['extendedIngredients']):
    ing_name = example_recipe_info['extendedIngredients'][index]['name']
    ing_list.append(ing_name)
print("Ingredients List:", ing_list, '\n')

# Ingredient types
ing_type_list = []
for index, val in enumerate(example_recipe_info['extendedIngredients']):
    ing_type = example_recipe_info['extendedIngredients'][index]['aisle']
    ing_type_list.append(ing_type)
ing_type_counts = dict(zip(Counter(ing_type_list).keys(), Counter(ing_type_list).values()))
print("Count of Ingredient Types:", ing_type_counts, '\n')

# Num steps
num_steps = len(example_recipe_info['analyzedInstructions'][0]['steps'])
print("Number of Steps:", num_steps, '\n')

# Num words in instructions
num_words = len(example_recipe_info['instructions'].split())
print("Number of Words in Instructions:", num_words, '\n')

Number of ingredients: 15 

Ingredients List: ['ancho chile powder', 'avocado', 'black beans', 'carrot', 'chile powder', 'cumin', 'garlic', 'ground flaxseed', 'kale', 'lemon juice', 'lime juice', 'quinoa', 'red bell pepper', 'salt', 'vegetable broth'] 

Count of Ingredient Types: {'Ethnic Foods;Spices and Seasonings': 1, 'Produce': 7, 'Pasta and Rice;Canned and Jarred': 1, 'Spices and Seasonings': 3, 'Health Foods;Baking': 1, 'Pasta and Rice;Health Foods': 1, 'Canned and Jarred': 1} 

Number of Steps: 8 

Number of Words in Instructions: 186 



In [14]:
# Nutrition
nutrients = []
nutrient_amts = []
for index, val in enumerate(example_recipe_info['nutrition']['nutrients']):
    nutrient_name = example_recipe_info['nutrition']['nutrients'][index]['title']
    nutrients.append(nutrient_name)
    nutrient_amt = example_recipe_info['nutrition']['nutrients'][index]['amount']
    nutrient_amts.append(nutrient_amt)
nutrition_info = dict(zip(nutrients, nutrient_amts))
print("Nutrition:", nutrition_info, '\n')
                                          
# Diets
diets = example_recipe_info['diets']
print("Diets:", diets, '\n')

Nutrition: {'Calories': 242.6, 'Fat': 7.38, 'Saturated Fat': 1.03, 'Carbohydrates': 37.19, 'Sugar': 1.8, 'Cholesterol': 0.0, 'Sodium': 250.66, 'Protein': 9.79, 'Vitamin K': 142.68, 'Vitamin A': 4183.66, 'Vitamin C': 48.48, 'Manganese': 1.01, 'Folate': 160.25, 'Fiber': 8.84, 'Copper': 0.63, 'Magnesium': 111.48, 'Phosphorus': 239.17, 'Vitamin B6': 0.38, 'Potassium': 667.61, 'Vitamin B1': 0.28, 'Iron': 2.98, 'Vitamin B2': 0.21, 'Vitamin E': 1.82, 'Zinc': 1.8, 'Vitamin B5': 0.91, 'Vitamin B3': 1.72, 'Calcium': 69.53, 'Selenium': 3.59} 

Diets: ['gluten free', 'dairy free', 'lacto ovo vegetarian', 'vegan'] 



In [15]:
#How long it takes to prepare the recipe.
print('Recipe Total Time (In Minutes):', example_recipe_info['readyInMinutes'], '\n')

print('Prep Time:', example_recipe_info['preparationMinutes'], '\n')

print('Cooking Time:', example_recipe_info['cookingMinutes'], '\n')

Recipe Total Time (In Minutes): 50 

Prep Time: 25 

Cooking Time: 25 



In [16]:
#Number of "likes" for this recipe.
print('Recipe Number Of Likes')
example_recipe_info['aggregateLikes']

Recipe Number Of Likes


50078

In [17]:
#Spoonacular's "score" for a particular recipe.
print('Recipe Spoonacular Score (0-100)')
example_recipe_info['spoonacularScore']

Recipe Spoonacular Score (0-100)


100.0

In [18]:
#Price per serving (in cents)
print('Price Per Serving (In Cents)')
example_recipe_info['pricePerServing']

Price Per Serving (In Cents)


101.23

In [19]:
print('Number Of Servings')
example_recipe_info['servings']

Number Of Servings


6

In [20]:
#It looks like Spoonacular is measuring "price" in cents.
#We need to divide by 100 to get price in dollars.
total_price = example_recipe_info['servings'] * example_recipe_info['pricePerServing']
print('Total Price For Recipe (In Dollars)')
round(total_price / 100,2)

Total Price For Recipe (In Dollars)


6.07

In [21]:
response_1_price = requests.get('https://api.spoonacular.com/recipes/592479/priceBreakdownWidget.json?&apiKey=c6943e3e749f4d5c8627c071b32cd36a')
example_recipe_price = response_1_price.json()
example_recipe_price

{'ingredients': [{'name': 'ancho chile powder',
   'image': 'chili-powder.jpg',
   'price': 24.21,
   'amount': {'metric': {'value': 1.0, 'unit': 'tsp'},
    'us': {'value': 1.0, 'unit': 'tsp'}}},
  {'name': 'avocado',
   'image': 'avocado.jpg',
   'price': 150.0,
   'amount': {'metric': {'value': 1.0, 'unit': ''},
    'us': {'value': 1.0, 'unit': ''}}},
  {'name': 'canned black beans',
   'image': 'black-beans.jpg',
   'price': 39.62,
   'amount': {'metric': {'value': 258.0, 'unit': 'g'},
    'us': {'value': 1.5, 'unit': 'cups'}}},
  {'name': 'carrot',
   'image': 'sliced-carrot.png',
   'price': 11.24,
   'amount': {'metric': {'value': 64.0, 'unit': 'g'},
    'us': {'value': 0.5, 'unit': 'cup'}}},
  {'name': 'chile powder',
   'image': 'chili-powder.jpg',
   'price': 2.14,
   'amount': {'metric': {'value': 0.25, 'unit': 'tsps'},
    'us': {'value': 0.25, 'unit': 'tsps'}}},
  {'name': 'cumin',
   'image': 'ground-cumin.jpg',
   'price': 6.61,
   'amount': {'metric': {'value': 0.5, 'un

In [22]:
#confirm that data is consistent for price - looks good.
round(example_recipe_price['totalCost'] / 100,2)

6.07

In [23]:
#Let's see what the /recipes/{id}/summary endpoint gets us.
#It looks like just a messy verbal description combining all the
#raw data in the /recipes/{id}/information endpoint.

response_1_summary = requests.get('https://api.spoonacular.com/recipes/592479/summary?&apiKey=c6943e3e749f4d5c8627c071b32cd36a')
example_recipe_summary = response_1_summary.json()
example_recipe_summary

{'id': 592479,
 'title': 'Kale and Quinoa Salad with Black Beans',
 'summary': 'Kale and Quinoa Salad with Black Beans is a side dish that serves 6. One portion of this dish contains approximately <b>10g of protein</b>, <b>7g of fat</b>, and a total of <b>243 calories</b>. For <b>$1.01 per serving</b>, this recipe <b>covers 24%</b> of your daily requirements of vitamins and minerals. 50078 people were impressed by this recipe. Head to the store and pick up vegetable broth, avocado, cumin, and a few other things to make it today. It is brought to you by blog.fatfreevegan.com. It is a good option if you\'re following a <b>gluten free, dairy free, lacto ovo vegetarian, and vegan</b> diet. From preparation to the plate, this recipe takes about <b>50 minutes</b>. Overall, this recipe earns a <b>tremendous spoonacular score of 100%</b>. Try <a href="https://spoonacular.com/recipes/skip-the-rice-and-try-quinoa-mexican-quinoa-salad-with-black-beans-and-corn-530227">Skip the Rice and Try Quinoa

In [24]:
#cleaning up this response ^
example_recipe_summary['summary'].replace('</b>','').replace('<b>','').replace('<', '').replace('>','').replace('//','').replace('href="https:spoonacular.com/recipes/skip-the-rice-and-try-quinoa-mexican-quinoa-salad-with-black-beans-and-corn-530227"','').replace('href="https:spoonacular.com/recipes/black-quinoa-kale-salad-15436"','').replace(' Try a Skip the Rice and Try Quinoa – Mexican Quinoa Salad with Black Beans and Corn/a, a Black Quinoa & Kale Salad/a, and a href="https:spoonacular.com/recipes/black-quinoa-kale-salad-42466"Black Quinoa & Kale Salad/a for similar recipes.','')

"Kale and Quinoa Salad with Black Beans is a side dish that serves 6. One portion of this dish contains approximately 10g of protein, 7g of fat, and a total of 243 calories. For $1.01 per serving, this recipe covers 24% of your daily requirements of vitamins and minerals. 50078 people were impressed by this recipe. Head to the store and pick up vegetable broth, avocado, cumin, and a few other things to make it today. It is brought to you by blog.fatfreevegan.com. It is a good option if you're following a gluten free, dairy free, lacto ovo vegetarian, and vegan diet. From preparation to the plate, this recipe takes about 50 minutes. Overall, this recipe earns a tremendous spoonacular score of 100%."

### 5. Call All Project Data From Spoonacular API

#### Get all recipes that we are interested in studying

Which recipes do we want to study?<br>
Which ingredients in particular are we interested in?<br>
What calorie thresholds are we interested in?<br>
...<br>
...<br>
???

In [25]:
diet_type = 'primal'
api_key = 'c6943e3e749f4d5c8627c071b32cd36a'
num = 100 # max number recipes per call is 100 
offs = 0 # number results to skip, increment each loop by 100 if number returned = 100
url = f'https://api.spoonacular.com/recipes/search?diet={diet_type}&offset={offs}&number={num}&apiKey={api_key}'
response = requests.get(url)
result = response.json()

In [26]:
df1 = pd.DataFrame()
df1

""


In [27]:
result

{'results': [{'id': 667917,
   'title': 'Cilantro Salsa',
   'readyInMinutes': 20,
   'servings': 2,
   'image': 'Cilantro-Salsa-667917.jpg',
   'imageUrls': ['Cilantro-Salsa-667917.jpg']},
  {'id': 685861,
   'title': 'Roasted Spaghetti Squash with Parmesan and Mushrooms',
   'readyInMinutes': 45,
   'servings': 2,
   'image': 'Roasted-Spaghetti-Squash-with-Parmesan-and-Mushrooms-685861.jpg',
   'imageUrls': ['Roasted-Spaghetti-Squash-with-Parmesan-and-Mushrooms-685861.jpg']},
  {'id': 474094,
   'title': 'Skinny Hot Spinach and Artichoke Dip with Fontina Cheese',
   'readyInMinutes': 25,
   'servings': 12,
   'image': 'Skinny-Hot-Spinach-and-Artichoke-Dip-with-Fontina-Cheese-474094.jpg',
   'imageUrls': ['Skinny-Hot-Spinach-and-Artichoke-Dip-with-Fontina-Cheese-474094.jpg']},
  {'id': 547998,
   'title': 'Grapefruit, Orange, and Avocado Salad',
   'readyInMinutes': 45,
   'servings': 4,
   'image': 'Grapefruit--Orange--and-Avocado-Salad-547998.jpg',
   'imageUrls': ['Grapefruit--Oran

In [28]:
len(result['results'])

100

In [29]:
result['number']

100

In [30]:
df = pd.DataFrame(result['results'])
df.head()
df.columns

Index(['id', 'title', 'readyInMinutes', 'servings', 'image', 'imageUrls'], dtype='object')

In [31]:
# this function allows us to retrieve a list of basic recipe info for each diet

diets = ['glutenFree', 'ketogenic', 'vegetarian', 'vegan', 'whole30', 
         'pescatarian', 'primal', 'paleo']

diets_test = ['primal', 'paleo']

def api_call(diet_list):
    api_key = 'c6943e3e749f4d5c8627c071b32cd36a'
    num = 100 # max number recipes per call is 100 
    all_recipes = []
    offs = 0 # number results to skip, increment each loop by 100 if number returned = 100
    for item in diet_list:
        diet_recipes = []
        url = f'https://api.spoonacular.com/recipes/search?diet={item}&offset={offs}&number={num}&apiKey={api_key}'
        response = requests.get(url)
        diet_recipe_call = response.json()
        diet_recipes.append(diet_recipe_call['results']) # append results
        while (diet_recipe_call['number'] == 100) & (offs < 500): # count recipes actually returned
            offs += 100 # offset results in next call
            response = requests.get(url)
            diet_recipe_call = response.json()
            diet_recipes.append(diet_recipe_call['results']) # append results
        return diet_recipes
    all_recipes.append(diet_recipes)
    return all_recipes

In [32]:
# collect recipe info in a dataframe object
api_call_test_results = api_call(diets_test)

In [33]:
df = pd.DataFrame(columns = ['id', 'title', 'readyInMinutes', 'servings', 'image', 'imageUrls'])
df

,id,title,readyInMinutes,servings,image,imageUrls


In [34]:
for index, val in enumerate(api_call_test_results):
    call_df = pd.DataFrame(api_call_test_results[index])
    df = pd.concat([df, call_df])
df.head()

,id,title,readyInMinutes,servings,image,imageUrls
0,667917,Cilantro Salsa,20,2,Cilantro-Salsa-667917.jpg,[Cilantro-Salsa-667917.jpg]
1,685861,Roasted Spaghetti Squash with Parmesan and Mus...,45,2,Roasted-Spaghetti-Squash-with-Parmesan-and-Mus...,[Roasted-Spaghetti-Squash-with-Parmesan-and-Mu...
2,474094,Skinny Hot Spinach and Artichoke Dip with Font...,25,12,Skinny-Hot-Spinach-and-Artichoke-Dip-with-Font...,[Skinny-Hot-Spinach-and-Artichoke-Dip-with-Fon...
3,547998,"Grapefruit, Orange, and Avocado Salad",45,4,Grapefruit--Orange--and-Avocado-Salad-547998.jpg,[Grapefruit--Orange--and-Avocado-Salad-547998....
4,551377,Toasted Almond Hazelnut Butter,45,4,Toasted-Almond-Hazelnut-Butter-551377.jpg,[Toasted-Almond-Hazelnut-Butter-551377.jpg]


In [35]:
# 300 recipes each of primal and paleo diets
len(df)

600

In [36]:
# 500 duplicates. not an efficient way to make calls
df.duplicated('id').value_counts()

True     500
False    100
dtype: int64

In [37]:
# get detailed info for each recipe returned with another function - one call per recipe
def recipe_id_api_call(recipe_ids_list):
    api_key = '36be87b928154489b9190320a260d6ee'
    all_recipes = []
    for recipe_id in recipe_ids_list:
        url = f'https://api.spoonacular.com/recipes/{recipe_id}/information?includeNutrition=true&apiKey={api_key}'
        response = requests.get(url)
        recipe_call = response.json()
        all_recipes.append(recipe_call)
    return all_recipes

In [38]:
recipe_list_test = [667917, 685861]
test_res = recipe_id_api_call(recipe_list_test)

In [41]:
# Try data cleaning protocol on two recipes called

# columns = ['vegetarian', 'vegan', 'glutenFree', 'dairyFree', 'veryHealthy', 'cheap', 
#            'veryPopular', 'sustainable', 'ketogenic', 'whole30', 'aggregateLikes', 
#            'spoonacularScore', 'healthScore', 'pricePerServing', 'extendedIngredients', 
#            'id', 'title', 'readyInMinutes', 'servings', 'nutrition', 'cuisines', 'dishTypes',
#            'diets', 'occasions', 'winePairing', 'instructions', 'analyzedInstructions', 
#            'preparationMinutes', 'cookingMinutes']

for recipe in test_res:
    # Add clean nutrition column
    nutrients = []
    nutrient_amts = []
    for index, val in enumerate(recipe['nutrition']['nutrients']):
        nutrient_name = recipe['nutrition']['nutrients'][index]['title']
        nutrients.append(nutrient_name)
        nutrient_amt = recipe['nutrition']['nutrients'][index]['amount']
        nutrient_amts.append(nutrient_amt)
    recipe['nutrition_info'] = dict(zip(nutrients, nutrient_amts))
    
    # Add num ingredients
    recipe['num_ingredients'] = len(recipe['extendedIngredients'])

    # Add list ingredients -- try overwriting existing information
    ing_list = []
    for index, val in enumerate(recipe['extendedIngredients']):
        ing_name = recipe['extendedIngredients'][index]['name']
        ing_list.append(ing_name)
    recipe['extendedIngredients'] = ing_list

    # Add ingredient types
    ing_type_list = []
    for index, val in enumerate(recipe['extendedIngredients']):
        ing_type = recipe['extendedIngredients'][index]['aisle']
        ing_type_list.append(ing_type)
    ing_type_counts = dict(zip(Counter(ing_type_list).keys(), Counter(ing_type_list).values()))
    recipe['ingredient_types'] = ing_type_counts

    # Num steps
    num_steps = len(recipe['analyzedInstructions'][0]['steps'])
    recipe['num_steps'] = num_steps

    # Num words in instructions
    num_words = len(recipe['instructions'].split())
    recipe['num_instruction_words'] = num_words
    
df = pd.DataFrame(test_res)
df

TypeError: string indices must be integers